# Segment Data from Poly detection

https://www.kaggle.com/competitions/bkai-igh-neopolyp/overview

## 1. Imports and Setup

#### 1.1 Imports

import all the required packages

In [1]:
import os
import glob
import pandas as pd
import cv2
from collections import Counter
import numpy as np

In [2]:
import segmentation_models as sm
import tensorflow as tf

Segmentation Models: using `keras` framework.


#### 1.2 Setup 

all the paths and where to find the data

In [3]:
data_src = r'C:\data\bkai-igh-neopolyp'
train_im_path = os.path.join(data_src, 'train')
masks_path = os.path.join(data_src, 'train_gt')
test_im_files = os.path.join(data_src, 'test')
csv_subm_data = os.path.join(data_src, 'sample_submission.csv')

In [4]:
BACKBONE = 'efficientnetb3'
BATCH_SIZE = 8
CLASSES = ['car', 'pedestrian']
LR = 0.0001
EPOCHS = 40

preprocess_input = sm.get_preprocessing(BACKBONE)

## 2. Data (EDA, Preprocessing)

#### 2.1 Read data from disk

In [5]:
train_ims = glob.glob(os.path.join(train_im_path, '**/*.jpeg'), recursive=True)
masks = glob.glob(os.path.join(masks_path, '**/*.jpeg'), recursive=True)

In [6]:
print('found {0} train images and {1} mask images'.format(len(train_ims), len(masks)))

found 1000 train images and 1000 mask images


#### 2.2 Extract all the images sizes

In [7]:
heights = []
widths = []

for im_path in train_ims:
    tmp_im = cv2.imread(im_path)
    h = tmp_im.shape[0]
    w = tmp_im.shape[1]
    heights.append(h)
    widths.append(w)

In [8]:
heights_occ = Counter(heights)
widths_occ = Counter(widths)

print('found heights:', heights_occ)
print('found widths:', widths_occ)

found heights: Counter({995: 589, 959: 261, 977: 127, 969: 19, 1024: 4})
found widths: Counter({1280: 1000})


## 3. Model Building

#### 3.1 Build U-Net to get started

In [9]:
# define network parameters
n_classes = 1 if len(CLASSES) == 1 else (len(CLASSES) + 1)  # case for binary and multiclass segmentation
activation = 'sigmoid' if n_classes == 1 else 'softmax'

#create model
model = sm.Unet(BACKBONE, classes=n_classes, activation=activation)

44107200/44107200 [==============================] - 3s 0us/step


In [10]:
# define optomizer
optim = tf.keras.optimizers.Adam(LR)

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
# set class weights for dice_loss (car: 1.; pedestrian: 2.; background: 0.5;)
dice_loss = sm.losses.DiceLoss(class_weights=np.array([1, 2, 0.5])) 
focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

# actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
# total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss 

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

# compile keras model with defined optimozer, loss and metrics
model.compile(optim, total_loss, metrics)

NameError: name 'np' is not defined